In [2]:
cd ../

/Users/arbaaz/Downloads/mabe/mabe_code


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
from data_utils import (
    get_index, 
    get_file_names,
    get_paths,
    get_csv,
    get_parqt,
    get_col_names, 
    sleep
)


path = "/Users/arbaaz/Downloads/mabe/ds"
get_file_names(path)

['sample_submission.csv',
 'test.csv',
 'test_tracking',
 'train.csv',
 'train_annotation',
 'train_tracking']

In [121]:
train_metadata = get_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [4]:
train_metadata["lab_id"].unique().tolist()

['AdaptableSnail',
 'BoisterousParrot',
 'CRIM13',
 'CalMS21_supplemental',
 'CalMS21_task1',
 'CalMS21_task2',
 'CautiousGiraffe',
 'DeliriousFly',
 'ElegantMink',
 'GroovyShrew',
 'InvincibleJellyfish',
 'JovialSwallow',
 'LyricalHare',
 'MABe22_keypoints',
 'MABe22_movies',
 'NiftyGoldfinch',
 'PleasantMeerkat',
 'ReflectiveManatee',
 'SparklingTapir',
 'TranquilPanther',
 'UppityFerret']

In [5]:
cols = ["mouse1_strain", "mouse2_strain", "mouse3_strain", "mouse4_strain"]
for col in cols:
    print(train_metadata[col].unique())
    print()

['CD-1 (ICR)' 'C57Bl/6N' 'C57Bl/6J' '129/SvEvTac' 'C57Bl/6J x Ai148'
 'BTBR' 'CD1' 'CFW']

['CD-1 (ICR)' 'C57Bl/6N' 'BALB/c' 'C57Bl/6J' '129/SvEvTac' 'CFW' 'BTBR'
 'CD1']

['CD-1 (ICR)' nan 'BTBR' 'C57Bl/6J']

['CD-1 (ICR)' nan]



In [6]:
cols = get_col_names("_color")

for col in cols:
    #print(train_metadata[col].unique())
    print(train_metadata[col].value_counts())
    print()

mouse1_color
black            5418
black and tan    3262
white             103
brown               7
Name: count, dtype: int64

mouse2_color
black            4832
black and tan    3262
white             692
brown               4
Name: count, dtype: int64

mouse3_color
black            4664
black and tan    3262
white              17
Name: count, dtype: int64

mouse4_color
white    10
Name: count, dtype: int64



In [7]:
cols = get_col_names("_sex")

for col in cols:
    print(train_metadata[col].value_counts())
    print()

mouse1_sex
male      8729
female      50
Name: count, dtype: int64

mouse2_sex
male      8423
female     319
Name: count, dtype: int64

mouse3_sex
male    7943
Name: count, dtype: int64

mouse4_sex
male    10
Name: count, dtype: int64



In [8]:
cols = get_col_names("_id")

for c in cols:
    print(train_metadata[c].value_counts())
    print()
    print(train_metadata[c].unique())
    print()

mouse1_id
6.0     48
4.0     44
5.0     40
3.0     35
9.0     31
        ..
80.0     2
62.0     2
77.0     2
76.0     2
79.0     2
Name: count, Length: 80, dtype: int64

[10.  3.  7. 11. 14. 12.  6.  5.  1.  4.  2. 55. 72. 42. 60. 19. 36. 74.
 70. 65. 59. 68. 75. 40. 49. 67. 69. 73. 37. 32. 76. 29. 54. 56. 39. 58.
 62. 46. 18. 35. 34. 52. 28. 20. 38. 63. 53. 80. 30. 13. 23. 66.  9. 77.
 71. 25. 47. 27. 22. 31. 79. 48. 61.  8. 33. 41. 17. 43. 51. 64. 50. 45.
 15. 44. 26. 57. 21. 24. 78. 16. nan]

mouse2_id
12.0    9
16.0    8
20.0    8
17.0    7
18.0    7
15.0    6
8.0     4
10.0    4
28.0    4
2.0     3
14.0    3
24.0    3
13.0    3
19.0    3
26.0    3
4.0     2
21.0    2
44.0    2
25.0    2
5.0     2
42.0    2
40.0    2
50.0    2
6.0     2
53.0    1
35.0    1
38.0    1
47.0    1
33.0    1
31.0    1
7.0     1
1.0     1
3.0     1
9.0     1
22.0    1
48.0    1
30.0    1
36.0    1
32.0    1
Name: count, dtype: int64

[24. 17. 21. 25. 28. 26. 20. 19. 15. 18. 12.  9.  7. 10. nan 50. 36. 14.

In [9]:
for c in get_col_names("_age"):
    print(train_metadata[c].value_counts())
    print()

mouse1_age
10-20 weeks    7926
> 40 weeks       53
17-21 weeks      52
8-12 weeks       50
8-26 weeks       24
8 weeks          21
7-14 weeks        8
12 weeks          6
Name: count, dtype: int64

mouse2_age
10-20 weeks    7926
6-12 weeks       53
8-13 weeks       52
8-12 weeks       50
8-57 weeks       24
4-6 weeks        21
7-14 weeks        8
12 weeks          6
Name: count, dtype: int64

mouse3_age
10-20 weeks    7926
8-12 weeks       17
Name: count, dtype: int64

mouse4_age
8-12 weeks    11
Name: count, dtype: int64



In [10]:
for c in get_col_names("_condition"):
    print(train_metadata[c].value_counts())
    print()

mouse1_condition
cabled device                122
cable                         92
estrus                        25
wireless device               20
day 2- 23:00 (lights off)     19
                            ... 
day 2- 22:04 (lights off)      1
day 2- 05:49 (lights off)      1
day 4- 01:51 (lights off)      1
day 4- 03:50 (lights off)      1
day 0- 09:40 (lights on)       1
Name: count, Length: 3752, dtype: int64

mouse2_condition
wireless device              20
day 2- 23:00 (lights off)    19
hormone primed               15
cabled device                15
day 4- 00:00 (lights off)    15
                             ..
day 0- 09:19 (lights on)      1
day 3- 17:27 (lights on)      1
day 2- 01:53 (lights off)     1
day 0- 20:15 (lights off)     1
day 2- 03:21 (lights off)     1
Name: count, Length: 3748, dtype: int64

mouse3_condition
day 2- 23:00 (lights off)    19
day 4- 00:00 (lights off)    15
day 2- 03:00 (lights off)    14
day 4- 03:00 (lights off)    14
day 1- 05:00 (lights off

In [11]:
train_metadata["frames_per_second"].value_counts()

frames_per_second
30.00     8548
25.00       57
10.00       52
120.00      36
45.00       36
60.00       18
50.00       17
24.99       14
60.01        5
24.98        3
120.02       2
24.62        1
24.36        1
Name: count, dtype: int64

In [12]:
train_metadata["frames_per_second"].mean(), train_metadata["frames_per_second"].std()

(30.40601478953356, 6.441435088939598)

In [13]:
train_metadata["video_duration_sec"].value_counts()

video_duration_sec
60.0     7927
600.0      59
900.0      53
600.1      18
600.2      13
         ... 
235.1       1
909.8       1
591.5       1
330.8       1
600.3       1
Name: count, Length: 635, dtype: int64

In [14]:
train_metadata["video_duration_sec"].mean(), train_metadata["video_duration_sec"].std()

(134.67042093287827, 637.0911103356581)

In [15]:
train_metadata["pix_per_cm_approx"].value_counts()

pix_per_cm_approx
11.70    7927
18.33     474
15.30      52
32.00      44
15.80      36
         ... 
19.00       1
49.00       1
58.00       1
63.00       1
56.00       1
Name: count, Length: 62, dtype: int64

In [16]:
train_metadata["pix_per_cm_approx"].mean(), train_metadata["pix_per_cm_approx"].std()

(12.43936518771331, 3.42677313048401)

In [17]:
train_metadata["video_width_pix"].value_counts()

video_width_pix
800     7926
1024     491
400       53
640       42
560       24
        ... 
404        1
545        1
406        1
408        1
734        1
Name: count, Length: 70, dtype: int64

In [18]:
train_metadata["video_width_pix"].mean(), train_metadata["video_width_pix"].std()

(813.6845278725825, 109.44153074240984)

In [19]:
col = "video_height_pix"

print(train_metadata[col].value_counts())
print()
train_metadata[col].mean(), train_metadata[col].std()

video_height_pix
800     7926
570      474
480      112
680       24
492       20
        ... 
309        1
1226       1
310        1
2014       1
301        1
Name: count, Length: 67, dtype: int64



(790.4981797497156, 134.40335677808915)

In [20]:
col = "arena_width_cm"

print(train_metadata[col].value_counts())
print()
train_metadata[col].mean(), train_metadata[col].std()

arena_width_cm
52.0     7926
35.0      540
19.0       53
15.0       52
33.0       47
120.0      36
28.0       33
38.0       27
30.8       24
60.0       17
12.7       11
50.0       10
45.0        8
45.7        6
Name: count, dtype: int64



(50.47703071672355, 7.5563815627102)

In [21]:
col = "arena_height_cm"

print(train_metadata[col].value_counts())
print()
train_metadata[col].mean(), train_metadata[col].std()

arena_height_cm
52.0     7926
15.0      495
28.5       53
35.0       52
20.0       45
120.0      36
18.0       33
19.0       28
22.2       24
38.0       21
60.0       17
22.0       11
46.0       11
50.0       10
45.0        8
16.0        8
18.5        6
12.0        6
Name: count, dtype: int64



(49.32005688282138, 10.687761717836116)

In [22]:
col = "arena_shape"
print(train_metadata[col].value_counts())

arena_shape
square               7961
rectangular           772
split rectangluar      36
circular               21
Name: count, dtype: int64


In [23]:
col = "arena_type"
print(train_metadata[col].value_counts())

arena_type
neutral                8017
resident-intruder       692
divided territories      36
familiar                 17
CSDS                     17
Name: count, dtype: int64


In [24]:
col = "body_parts_tracked"
print(train_metadata[col].value_counts())

body_parts_tracked
["body_center", "ear_left", "ear_right", "forepaw_left", "forepaw_right", "hindpaw_left", "hindpaw_right", "neck", "nose", "tail_base", "tail_midpoint", "tail_tip"]                                                                                                                                                                                               7926
["ear_left", "ear_right", "hip_left", "hip_right", "neck", "nose", "tail_base"]                                                                                                                                                                                                                                                                                     634
["body_center", "ear_left", "ear_right", "lateral_left", "lateral_right", "nose", "tail_base"]                                                                                                                                                                       

In [25]:
col = "behaviors_labeled"
print(train_metadata[col].value_counts())

behaviors_labeled
["mouse1,mouse2,attack", "mouse1,mouse2,mount", "mouse1,mouse2,sniff"]                                                                                                                                                                        80
["mouse1,mouse2,sniff", "mouse1,mouse2,sniffgenital"]                                                                                                                                                                                         74
["mouse1,mouse2,attack", "mouse1,mouse2,chase", "mouse1,mouse2,sniff"]                                                                                                                                                                        52
["mouse1,mouse2,mount", "mouse1,mouse2,sniff", "mouse1,mouse2,sniffgenital"]                                                                                                                                                                  36
["mouse1,mouse2,al

In [26]:
print(train_metadata[col].unique())

['["mouse1,mouse2,approach", "mouse1,mouse2,attack", "mouse1,mouse2,avoid", "mouse1,mouse2,chase", "mouse1,mouse2,chaseattack", "mouse1,mouse2,submit", "mouse1,mouse3,approach", "mouse1,mouse3,attack", "mouse1,mouse3,avoid", "mouse1,mouse3,chase", "mouse1,mouse3,chaseattack", "mouse1,mouse3,submit", "mouse1,mouse4,approach", "mouse1,mouse4,attack", "mouse1,mouse4,avoid", "mouse1,mouse4,chase", "mouse1,mouse4,chaseattack", "mouse1,mouse4,submit", "mouse1,self,rear", "mouse2,mouse1,approach", "mouse2,mouse1,attack", "mouse2,mouse1,avoid", "mouse2,mouse1,chase", "mouse2,mouse1,chaseattack", "mouse2,mouse1,submit", "mouse2,mouse3,approach", "mouse2,mouse3,attack", "mouse2,mouse3,avoid", "mouse2,mouse3,chase", "mouse2,mouse3,chaseattack", "mouse2,mouse3,submit", "mouse2,mouse4,approach", "mouse2,mouse4,attack", "mouse2,mouse4,avoid", "mouse2,mouse4,chase", "mouse2,mouse4,chaseattack", "mouse2,mouse4,submit", "mouse2,self,rear", "mouse3,mouse1,approach", "mouse3,mouse1,attack", "mouse3,mouse

In [27]:
col = "tracking_method"
print(train_metadata[col].value_counts())

tracking_method
custom HRnet    7926
MARS             528
DeepLabCut       212
SLEAP            124
Name: count, dtype: int64


In [28]:
for lab_id, count in train_metadata.groupby("lab_id").size().sort_values(ascending=False).items():
    print(f"{lab_id}       {count}")

MABe22_keypoints       5320
MABe22_movies       2606
CalMS21_supplemental       297
CalMS21_task1       101
CalMS21_task2       76
SparklingTapir       70
JovialSwallow       52
PleasantMeerkat       36
InvincibleJellyfish       33
TranquilPanther       25
LyricalHare       24
UppityFerret       21
CRIM13       21
ReflectiveManatee       20
ElegantMink       19
GroovyShrew       17
AdaptableSnail       17
NiftyGoldfinch       11
CautiousGiraffe       10
BoisterousParrot       8
DeliriousFly       6


In [55]:
import ast

index = get_index(range(8000))
ast.literal_eval(train_metadata["body_parts_tracked"][index]), index

(['body_center',
  'ear_left',
  'ear_right',
  'forepaw_left',
  'forepaw_right',
  'hindpaw_left',
  'hindpaw_right',
  'neck',
  'nose',
  'tail_base',
  'tail_midpoint',
  'tail_tip'],
 1593)

In [60]:
train_metadata["behaviors_labeled"][0]

'["mouse1,mouse2,approach", "mouse1,mouse2,attack", "mouse1,mouse2,avoid", "mouse1,mouse2,chase", "mouse1,mouse2,chaseattack", "mouse1,mouse2,submit", "mouse1,mouse3,approach", "mouse1,mouse3,attack", "mouse1,mouse3,avoid", "mouse1,mouse3,chase", "mouse1,mouse3,chaseattack", "mouse1,mouse3,submit", "mouse1,mouse4,approach", "mouse1,mouse4,attack", "mouse1,mouse4,avoid", "mouse1,mouse4,chase", "mouse1,mouse4,chaseattack", "mouse1,mouse4,submit", "mouse1,self,rear", "mouse2,mouse1,approach", "mouse2,mouse1,attack", "mouse2,mouse1,avoid", "mouse2,mouse1,chase", "mouse2,mouse1,chaseattack", "mouse2,mouse1,submit", "mouse2,mouse3,approach", "mouse2,mouse3,attack", "mouse2,mouse3,avoid", "mouse2,mouse3,chase", "mouse2,mouse3,chaseattack", "mouse2,mouse3,submit", "mouse2,mouse4,approach", "mouse2,mouse4,attack", "mouse2,mouse4,avoid", "mouse2,mouse4,chase", "mouse2,mouse4,chaseattack", "mouse2,mouse4,submit", "mouse2,self,rear", "mouse3,mouse1,approach", "mouse3,mouse1,attack", "mouse3,mouse1

In [5]:
get_file_names(path)

['sample_submission.csv',
 'test.csv',
 'test_tracking',
 'train.csv',
 'train_annotation',
 'train_tracking']

In [6]:
dataset_names = get_file_names(path+"/train_tracking")
dataset_names

['AdaptableSnail',
 'BoisterousParrot',
 'CRIM13',
 'CalMS21_supplemental',
 'CalMS21_task1',
 'CalMS21_task2',
 'CautiousGiraffe',
 'DeliriousFly',
 'ElegantMink',
 'GroovyShrew',
 'InvincibleJellyfish',
 'JovialSwallow',
 'LyricalHare',
 'MABe22_keypoints',
 'MABe22_movies',
 'NiftyGoldfinch',
 'PleasantMeerkat',
 'ReflectiveManatee',
 'SparklingTapir',
 'TranquilPanther',
 'UppityFerret']

In [7]:
dataset_paths = get_paths(path+"/train_tracking")
dataset_paths

['/kaggle/input/MABe-mouse-behavior-detection/train_tracking/AdaptableSnail',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/BoisterousParrot',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/CRIM13',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/CalMS21_supplemental',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/CalMS21_task1',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/CalMS21_task2',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/CautiousGiraffe',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/DeliriousFly',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/ElegantMink',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/GroovyShrew',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/InvincibleJellyfish',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/JovialSwallow',
 '/kaggle/input/MABe-mouse-behavior-detection/train_tracking/LyricalH

In [27]:
videos = get_file_names(dataset_paths[0])
videos

['1212811043.parquet',
 '1260392287.parquet',
 '1351098077.parquet',
 '1408652858.parquet',
 '143861384.parquet',
 '1596473327.parquet',
 '1643942986.parquet',
 '1717182687.parquet',
 '2078515636.parquet',
 '209576908.parquet',
 '278643799.parquet',
 '351967631.parquet',
 '355542626.parquet',
 '44566106.parquet',
 '678426900.parquet',
 '705948978.parquet',
 '878123481.parquet']

In [29]:
snail_ds_path = os.path.join(dataset_paths[0], videos[0])
snail = get_parqt(snail_ds_path)
snail.head()

,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,496.187012,376.475006
1,0,1,ear_left,494.059998,343.924011
2,0,1,ear_right,518.765015,367.362000
3,0,1,lateral_left,474.536987,370.563995
4,0,1,lateral_right,505.825012,394.937012


In [30]:
for frame, grp in snail.groupby("video_frame"):
    print(f"{frame}  {grp}")
    break

0      video_frame  mouse_id       bodypart           x           y
0             0         1    body_center  496.187012  376.475006
1             0         1       ear_left  494.059998  343.924011
2             0         1      ear_right  518.765015  367.362000
3             0         1   lateral_left  474.536987  370.563995
4             0         1  lateral_right  505.825012  394.937012
5             0         1           nose  527.765015  342.686005
6             0         1      tail_base  477.536011  398.338013
7             0         1  tail_midpoint  446.834015  429.035004
8             0         1       tail_tip  424.618011  457.161011
9             0         2    body_center  702.916016  322.872986
10            0         2       ear_left  727.033020  302.700989
11            0         2      ear_right  740.577026  331.324005
12            0         2   lateral_left  698.749023  300.458008
13            0         2  lateral_right  702.254028  346.290009
14            0       

In [31]:
video_ids = [int(v.split(".")[0]) for v in videos]
video_ids

[1212811043,
 1260392287,
 1351098077,
 1408652858,
 143861384,
 1596473327,
 1643942986,
 1717182687,
 2078515636,
 209576908,
 278643799,
 351967631,
 355542626,
 44566106,
 678426900,
 705948978,
 878123481]

In [34]:
group = train_metadata.groupby("lab_id").get_group("AdaptableSnail")
grp_video_ids = group["video_id"].unique().tolist()
grp_video_ids

[44566106,
 143861384,
 209576908,
 278643799,
 351967631,
 355542626,
 678426900,
 705948978,
 878123481,
 1212811043,
 1260392287,
 1351098077,
 1408652858,
 1596473327,
 1643942986,
 1717182687,
 2078515636]

In [43]:
[id in grp_video_ids for id in video_ids]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [44]:
annotation_path = os.path.join(path, "train_annotation")
ds_annotation_names = get_file_names(annotation_path)
ds_annotation_names

['AdaptableSnail',
 'BoisterousParrot',
 'CRIM13',
 'CalMS21_supplemental',
 'CalMS21_task1',
 'CalMS21_task2',
 'CautiousGiraffe',
 'DeliriousFly',
 'ElegantMink',
 'GroovyShrew',
 'InvincibleJellyfish',
 'JovialSwallow',
 'LyricalHare',
 'NiftyGoldfinch',
 'PleasantMeerkat',
 'ReflectiveManatee',
 'SparklingTapir',
 'TranquilPanther',
 'UppityFerret']

In [46]:
ds_ann_path = os.path.join(annotation_path, ds_annotation_names[0])
ds_annotation_files = get_file_names(ds_ann_path)
ds_annotation_files

['1212811043.parquet',
 '1260392287.parquet',
 '1351098077.parquet',
 '1408652858.parquet',
 '143861384.parquet',
 '1596473327.parquet',
 '1643942986.parquet',
 '1717182687.parquet',
 '2078515636.parquet',
 '209576908.parquet',
 '278643799.parquet',
 '351967631.parquet',
 '355542626.parquet',
 '44566106.parquet',
 '678426900.parquet',
 '705948978.parquet',
 '878123481.parquet']

In [41]:
video_ids_ann = [int(a.split(".")[0]) for a in ds_annotation_files]
[x in video_ids for x in video_ids_ann]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [49]:
ann = get_parqt(os.path.join(ds_ann_path, ds_annotation_files[0]))
ann.head()

,agent_id,target_id,action,start_frame,stop_frame
0,1,3,chase,2,54
1,1,3,chase,128,234
2,3,2,avoid,324,342
3,3,1,avoid,324,342
4,1,2,chase,942,1052


In [50]:
ann

,agent_id,target_id,action,start_frame,stop_frame
0,1,3,chase,2,54
1,1,3,chase,128,234
2,3,2,avoid,324,342
3,3,1,avoid,324,342
4,1,2,chase,942,1052
...,...,...,...,...,...
365,1,2,chase,86668,86689
366,1,4,avoid,86815,86845
367,1,3,attack,89171,89194
368,4,2,avoid,89272,89299


In [51]:
ann["action"].unique()

array(['chase', 'avoid', 'attack', 'chaseattack'], dtype=object)

In [54]:
sample = get_csv(os.path.join(path, "sample_submission.csv"))
sample.head()

,row_id,video_id,agent_id,target_id,action,start_frame,stop_frame
0,0,438887472,mouse1,mouse2,sniff,0,1


In [60]:
test = get_parqt(os.path.join(path, "test_tracking/AdaptableSnail/438887472.parquet"))
test.head()

,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,909.577026,960.265991
1,0,1,ear_left,861.718018,988.614014
2,0,1,ear_right,882.953003,934.760010
3,0,1,headpiece_bottombackright,828.593994,997.226013
4,0,1,headpiece_bottomfrontleft,843.098022,948.182983


In [64]:
test_metadata = get_csv(os.path.join(path, "test.csv"))
test_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,438887472,CD-1 (ICR),white,male,13.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1214,1090,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [11]:
get_file_names(path)

['sample_submission.csv',
 'test.csv',
 'test_tracking',
 'train.csv',
 'train_annotation',
 'train_tracking']

In [26]:
get_parqt(os.path.join(path, "train_tracking")+"/AdaptableSnail"+"/1212811043.parquet")

,video_frame,mouse_id,bodypart,x,y
0,0,1,body_center,496.187012,376.475006
1,0,1,ear_left,494.059998,343.924011
2,0,1,ear_right,518.765015,367.362000
3,0,1,lateral_left,474.536987,370.563995
4,0,1,lateral_right,505.825012,394.937012
...,...,...,...,...,...
2677001,89975,4,lateral_right,348.324005,584.469971
2677002,89975,4,nose,420.062988,585.309998
2677003,89975,4,tail_base,323.226013,561.976990
2677004,89975,4,tail_midpoint,287.417999,552.995972


In [17]:
datasets_by_lab_id = {}

for id, grp in train_metadata.groupby("lab_id"):
    datasets_by_lab_id[id] = grp

In [30]:
datasets_by_tracking_method = {}

for k, v in datasets_by_lab_id.items():
    for method in v["tracking_method"].unique():
        if method not in datasets_by_tracking_method:
            datasets_by_tracking_method[method] = [k]
        else:
            datasets_by_tracking_method[method].append(k)
        
for k, v in datasets_by_tracking_method.items():
    for x in v:
        print(k, x)
    print()

DeepLabCut AdaptableSnail
DeepLabCut BoisterousParrot
DeepLabCut CautiousGiraffe
DeepLabCut DeliriousFly
DeepLabCut LyricalHare
DeepLabCut PleasantMeerkat
DeepLabCut ReflectiveManatee
DeepLabCut SparklingTapir
DeepLabCut UppityFerret

MARS CRIM13
MARS CalMS21_supplemental
MARS CalMS21_task1
MARS CalMS21_task2
MARS InvincibleJellyfish

SLEAP ElegantMink
SLEAP GroovyShrew
SLEAP JovialSwallow
SLEAP NiftyGoldfinch
SLEAP TranquilPanther

custom HRnet MABe22_keypoints
custom HRnet MABe22_movies



In [26]:
metadata_by_lab = train_metadata.groupby("lab_id")

body_parts_by_lab = {}
for id, grp in metadata_by_lab:
    body_parts_by_lab[grp["lab_id"].unique()[0]] = grp["body_parts_tracked"]


[k for k in body_parts_by_lab]

['AdaptableSnail',
 'BoisterousParrot',
 'CRIM13',
 'CalMS21_supplemental',
 'CalMS21_task1',
 'CalMS21_task2',
 'CautiousGiraffe',
 'DeliriousFly',
 'ElegantMink',
 'GroovyShrew',
 'InvincibleJellyfish',
 'JovialSwallow',
 'LyricalHare',
 'MABe22_keypoints',
 'MABe22_movies',
 'NiftyGoldfinch',
 'PleasantMeerkat',
 'ReflectiveManatee',
 'SparklingTapir',
 'TranquilPanther',
 'UppityFerret']

In [37]:
import json
from collections import defaultdict

body_parts_count_by_lab = defaultdict(lambda: defaultdict(int))

for lab_id, body_parts_tracked in body_parts_by_lab.items():
    for parts in body_parts_tracked:
        for p in json.loads(parts):
            body_parts_count_by_lab[lab_id][p] += 1
                

for lab_id, counts in body_parts_count_by_lab.items():
    print(f"Lab: {lab_id}")
    for part, count in counts.items():
        print(f"  {part}: {count}")
    print()


Lab: AdaptableSnail
  body_center: 17
  ear_left: 17
  ear_right: 17
  headpiece_bottombackleft: 7
  headpiece_bottombackright: 7
  headpiece_bottomfrontleft: 7
  headpiece_bottomfrontright: 7
  headpiece_topbackleft: 7
  headpiece_topbackright: 7
  headpiece_topfrontleft: 7
  headpiece_topfrontright: 7
  lateral_left: 17
  lateral_right: 17
  neck: 17
  nose: 17
  tail_base: 17
  tail_midpoint: 17
  tail_tip: 17

Lab: BoisterousParrot
  body_center: 8
  ear_left: 8
  ear_right: 8
  nose: 8
  tail_base: 8

Lab: CRIM13
  ear_left: 21
  ear_right: 21
  hip_left: 21
  hip_right: 21
  neck: 21
  nose: 21
  tail_base: 21

Lab: CalMS21_supplemental
  ear_left: 297
  ear_right: 297
  hip_left: 297
  hip_right: 297
  neck: 297
  nose: 297
  tail_base: 297

Lab: CalMS21_task1
  ear_left: 101
  ear_right: 101
  hip_left: 101
  hip_right: 101
  neck: 101
  nose: 101
  tail_base: 101

Lab: CalMS21_task2
  ear_left: 76
  ear_right: 76
  hip_left: 76
  hip_right: 76
  neck: 76
  nose: 76
  tail_base

In [40]:
# Assuming body_parts_count_by_lab is a dict of dicts

# Get a list of sets of body part names for each lab
body_part_sets = [set(parts.keys()) for parts in body_parts_count_by_lab.values()]

# Find intersection across all labs
common_body_parts = set.intersection(*body_part_sets)

print(f"Common body parts across all labs ({len(common_body_parts)}):")
for part in sorted(common_body_parts):
    print("  ", part)


Common body parts across all labs (3):
   ear_left
   ear_right
   tail_base


In [60]:
unique_body_parts = set()

for lab, body_parts_count in body_parts_count_by_lab.items():
    for body_parts in body_parts_count.keys():
        unique_body_parts.add(body_parts)
        
unique_body_parts

{'body_center',
 'ear_left',
 'ear_right',
 'forepaw_left',
 'forepaw_right',
 'head',
 'headpiece_bottombackleft',
 'headpiece_bottombackright',
 'headpiece_bottomfrontleft',
 'headpiece_bottomfrontright',
 'headpiece_topbackleft',
 'headpiece_topbackright',
 'headpiece_topfrontleft',
 'headpiece_topfrontright',
 'hindpaw_left',
 'hindpaw_right',
 'hip_left',
 'hip_right',
 'lateral_left',
 'lateral_right',
 'neck',
 'nose',
 'spine_1',
 'spine_2',
 'tail_base',
 'tail_middle_1',
 'tail_middle_2',
 'tail_midpoint',
 'tail_tip'}

In [ ]:
exclusive_body_parts_by_lab = []

for body_part in unique_body_parts:
    flag = 0
    lab_id = None
    
    for lab, body_parts_count in body_parts_count_by_lab.items():
        if body_part in body_parts_count.keys():
            flag += 1
            
            if flag == 1:
                lab_id = lab
        if flag > 1:
            break
        
    if flag == 1:
        exclusive_body_parts_by_lab.append((lab_id, body_part))
        
exclusive_body_parts_by_lab

[('GroovyShrew', 'head'),
 ('AdaptableSnail', 'headpiece_bottomfrontright'),
 ('AdaptableSnail', 'headpiece_bottomfrontleft'),
 ('UppityFerret', 'spine_2'),
 ('UppityFerret', 'tail_middle_2'),
 ('AdaptableSnail', 'headpiece_topbackleft'),
 ('AdaptableSnail', 'headpiece_topfrontleft'),
 ('AdaptableSnail', 'headpiece_bottombackright'),
 ('AdaptableSnail', 'headpiece_topfrontright'),
 ('UppityFerret', 'spine_1'),
 ('AdaptableSnail', 'headpiece_topbackright'),
 ('AdaptableSnail', 'headpiece_bottombackleft'),
 ('UppityFerret', 'tail_middle_1')]

In [106]:
labels_by_lab = defaultdict(set)

for lab_id, grp in train_metadata.groupby("lab_id"):
    for labels in grp["behaviors_labeled"]:
        try:
            pass
            #print(json.loads(labels))
        except Exception as e:
            print(labels)
            
        #for label in json.loads(labels):
        #    labels_by_lab[lab_id].add(label)



In [119]:
train_metadata["behaviors_labeled"].isna().sum()


np.int64(7941)

In [137]:
count = 0

for idx, row in train_metadata.iterrows():
    if pd.isna(row["behaviors_labeled"]):
        count += 1
count

7941

In [ ]:


for lab_id, labels in labels_by_lab.items():
    for label in labels:
        print(label)
        break
    break

["mouse1,mouse2,approach", "mouse1,mouse2,attack", "mouse1,mouse2,avoid", "mouse1,mouse2,chase", "mouse1,mouse2,chaseattack", "mouse1,mouse2,submit", "mouse1,mouse3,approach", "mouse1,mouse3,attack", "mouse1,mouse3,avoid", "mouse1,mouse3,chase", "mouse1,mouse3,chaseattack", "mouse1,mouse3,submit", "mouse1,mouse4,approach", "mouse1,mouse4,attack", "mouse1,mouse4,avoid", "mouse1,mouse4,chase", "mouse1,mouse4,chaseattack", "mouse1,mouse4,submit", "mouse1,self,rear", "mouse2,mouse1,approach", "mouse2,mouse1,attack", "mouse2,mouse1,avoid", "mouse2,mouse1,chase", "mouse2,mouse1,chaseattack", "mouse2,mouse1,submit", "mouse2,mouse3,approach", "mouse2,mouse3,attack", "mouse2,mouse3,avoid", "mouse2,mouse3,chase", "mouse2,mouse3,chaseattack", "mouse2,mouse3,submit", "mouse2,mouse4,approach", "mouse2,mouse4,attack", "mouse2,mouse4,avoid", "mouse2,mouse4,chase", "mouse2,mouse4,chaseattack", "mouse2,mouse4,submit", "mouse2,self,rear", "mouse3,mouse1,approach", "mouse3,mouse1,attack", "mouse3,mouse1,

In [152]:
train_metadata.shape[0] - train_metadata["behaviors_labeled"].isna().sum()

np.int64(848)

In [ ]:
annotation_path = path+"/train_annotation"
annotation_files = get_file_names(annotation_path)

['AdaptableSnail',
 'BoisterousParrot',
 'CRIM13',
 'CalMS21_supplemental',
 'CalMS21_task1',
 'CalMS21_task2',
 'CautiousGiraffe',
 'DeliriousFly',
 'ElegantMink',
 'GroovyShrew',
 'InvincibleJellyfish',
 'JovialSwallow',
 'LyricalHare',
 'NiftyGoldfinch',
 'PleasantMeerkat',
 'ReflectiveManatee',
 'SparklingTapir',
 'TranquilPanther',
 'UppityFerret']

In [177]:
for file in annotation_files:
    for pq_file in get_file_names(annotation_path+f"/{file}"):
        df = get_parqt(annotation_path+f"/{file}/{pq_file}")
        #print(df["action"].isna().any())

In [182]:
na_labs = defaultdict(int)

for idx, row in train_metadata.iterrows():
    if pd.isna(row["behaviors_labeled"]):
        na_labs[row["lab_id"]] += 1
 
na_labs

defaultdict(int,
            {'MABe22_keypoints': 5320,
             'MABe22_movies': 2606,
             'SparklingTapir': 15})